# CNN für NBA Spielererkennung

### Eigener Use Case / Robert Kurtz / July 2023

- Including Lebron James, Steph Curry, Giannis Antentokounmpo, Dennis Schröder &  Franz Wagner

## Idee:

Datensammlung: Ich suche Bilder der 5 Basketballspieler (Steph, Lebron, Giannis, Franz & Dennis Schröder) aus dem Internet oder anderen Quellen. Ich sorge dafür, dass ich genügend Bilder für Training und Test habe und achte darauf, dass die Bilder eine ausreichend hohe Auflösung haben und die Spieler gut sichtbar sind.

Daten vorbereiten: Ich organisiere die gesammelten Bilder in separate Ordner für jede Spielerkategorie. Ich teile die Daten in einen Trainings- und einen Testdatensatz auf und passe die Größe der Bilder an, um eine einheitliche Eingabegröße für das CNN-Modell zu gewährleisten.

CNN-Modell erstellen: Ich verwende TensorFlow oder Keras, um das CNN-Modell zu erstellen. Ich definiere die Architektur des Modells, einschließlich Convolutional Layers, Pooling Layers, Fully Connected Layers usw. Dabei sorge ich dafür, dass das Modell ausreichend komplex ist, um die Unterschiede zwischen den Spielern zu erkennen.

Modelltraining: Ich trainiere das CNN-Modell mit den vorbereiteten Trainingsdaten. Dazu wähle ich eine angemessene Anzahl von Epochen und eine passende Batch-Größe aus, um das Modell zu trainieren. Während des Trainings überwache ich den Fortschritt anhand von Metriken wie Genauigkeit und Verlust.

Modellbewertung: Nachdem das Modell trainiert wurde, werte ich es mit den vorbereiteten Testdaten aus, um seine Leistung zu bewerten. Ich überprüfe die Genauigkeit des Modells und analysiere, wie gut es die Spielerklassen erkennt.

Feinabstimmung (optional): Wenn das Modell nicht die gewünschte Leistung erzielt, kann ich versuchen, die Hyperparameter zu optimieren oder das Modell feinabstimmen, um die Ergebnisse zu verbessern.

Anwendung: Schließlich wende ich das trainierte CNN-Modell auf neue, nicht verwendete Bilder der 5 Basketballspieler an, um zu sehen, wie gut es sie klassifiziert.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [11]:
# Pfade zu den Datenordnern
data_dir = '/Users/robertkurtz/Desktop/DataScience/Datasets/AA Spielererkennung/data'  # Passe den Pfad an
train_data_dir = os.path.join(data_dir, 'train')
test_data_dir = os.path.join(data_dir, 'test')

# Parameter für Training
img_width, img_height = 224, 224
batch_size = 10
num_epochs = 8

# Erstelle einen ImageDataGenerator für Data Augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Passe Data Augmentation auf den Trainingsdatensatz an
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Lade das vortrainierte MobileNetV2-Modell ohne den Fully Connected Layer
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Füge einen eigenen Fully Connected Layer hinzu
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)

# Erstelle das Modell
model = Model(inputs=base_model.input, outputs=predictions)

# Friere die Gewichte des vortrainierten Teils des Modells ein
for layer in base_model.layers:
    layer.trainable = False

# Kompiliere das Modell
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Found 60 images belonging to 5 classes.
Epoch 1/8
6/6 [==============================] - 6s 597ms/step - loss: 2.9467 - accuracy: 0.2833
Epoch 2/8
6/6 [==============================] - 3s 593ms/step - loss: 1.9910 - accuracy: 0.4833
Epoch 3/8
6/6 [==============================] - 3s 553ms/step - loss: 1.1795 - accuracy: 0.4833
Epoch 4/8
6/6 [==============================] - 3s 580ms/step - loss: 0.6811 - accuracy: 0.7167
Epoch 5/8
6/6 [==============================] - 3s 358ms/step - loss: 0.4807 - accuracy: 0.8500
Epoch 6/8
6/6 [==============================] - 3s 583ms/step - loss: 0.3968 - accuracy: 0.9000
Epoch 7/8
6/6 [==============================] - 3s 563ms/step - loss: 0.3196 - accuracy: 0.9000
Epoch 8/8
6/6 [==============================] - 3s 599ms/step - loss: 0.3172 - accuracy: 0.9000


## Training

In [ ]:
# Training des Modells
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=num_epochs)

## Testing

In [12]:
# Passe Data Augmentation auf den Testdatensatz an
test_generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)  # Achtung: shuffle auf False setzen, damit die Reihenfolge beibehalten wird

# Evaluierung des Modells mit den Testdaten
test_loss, test_accuracy = model.evaluate(
    test_generator,
    steps=test_generator.samples // batch_size)

print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

Found 15 images belonging to 5 classes.
1/1 [==============================] - 1s 1s/step - loss: 0.8181 - accuracy: 0.8000
Test Loss: 0.8181015849113464, Test Accuracy: 0.800000011920929


## Erkennst du mich? Ich bin's Dennis!

- Test auf einem zusätzlichen Bild

In [15]:
from tensorflow.keras.preprocessing import image

# Pfade zu den Dateien und Bildern
test_image_path = '/Users/robertkurtz/Desktop/DataScience/Datasets/AA Spielererkennung/data/testX/DennisX.jpeg'  # Passe den Pfad entsprechend an
class_names = ['Dennis Schröder', 'Franz Wagner', 'Giannis Antetokounmpo', 'LeBron James', 'Stephen Curry']

# Lade das Bild und passe es an
img = image.load_img(test_image_path, target_size=(img_width, img_height))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Normalisiere die Pixelwerte auf den Bereich [0, 1]
img_array = np.expand_dims(img_array, axis=0)  # Füge eine Batch-Dimension hinzu

# Mache die Vorhersage
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)  # Finde den Index des höchsten Wahrscheinlichkeitswerts

# Gebe den erkannten Spielernamen aus
print(f"Erkannter Spieler: {class_names[predicted_class]}")


1/1 [==============================] - 0s 42ms/step
Erkannter Spieler: Dennis Schröder
